<a href="https://colab.research.google.com/github/LokeshVadlamudi/DeepLearningClass/blob/master/dl1mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importing the libraries

In [0]:
import numpy as np

In [0]:
from keras.datasets import mnist

#Loading the dataset

In [0]:
(x_train,y_train) , (x_test,y_test) = mnist.load_data()

#Normalizing the data inputs

In [0]:
images, labels = (x_train[0:1000].reshape(1000,28*28))/255,y_train[0:1000]

#One Hot encoding the output labels

In [0]:
one_hot_labels = np.zeros((len(labels),10))

In [0]:
for i,l in enumerate(labels):
  one_hot_labels[i][l] = 1
labels = one_hot_labels

In [0]:
test_images = x_test.reshape(len(x_test),28*28)/255
test_labels = np.zeros((len(y_test),10))


In [0]:
for i,l in enumerate(y_test):
  test_labels[i][l] = 1

#Used Different Activation functions ( added relu , tanh and softmax)

In [0]:
def tanh(x):
  return np.tanh(x)

In [0]:
def tanh2deriv(output):
  return 1 - (output ** 2)

In [0]:
def relu(x):
  return (x>=0)*x

In [0]:
def relu2deriv(x):
  return x>=0

In [0]:
def softmax(x):
  temp = np.exp(x)
  return temp / np.sum(temp,axis = 1, keepdims = True)

#Number of iterations, and different hidden layer sizes

In [0]:
iterations , hidden_size, hidden_size1 = (300,200,100)

#different learning rates for different layers

In [0]:
alpha1,alpha2,alpha3 = 2,3,1.5

#Initial input and number of output labels

In [0]:
inputPixels , num_labels = (784,10)

#batch size for stochastic mini-batch gradient descent

In [0]:
batch_size = 100

#Weights between layer_0 and layer_1

In [0]:
weights_0_1 = 0.02*np.random.random((inputPixels,hidden_size))-0.01

#weights between layer_1 and layer_2

In [0]:
weights_1_2 = 0.2*np.random.random((hidden_size,hidden_size1))-0.09

#Added additional weights for extra layer

In [0]:
weights_2_3 = 0.3 * np.random.random((hidden_size1,num_labels)) - 0.1

#Added an extra layer to make it three layer Neural Network.

In [215]:
for j in range(iterations):
  correct_cnt = 0
  for i in range(int(len(images) / batch_size)):
    batch_start, batch_end = ((i * batch_size), ((i + 1)*batch_size))
    layer_0 = images[batch_start:batch_end]

    layer_1 = tanh(np.dot(layer_0,weights_0_1))

    #drop out mask on layer 1
    dropout_mask = np.random.randint(2,size=layer_1.shape)
    layer_1 *= dropout_mask*2

    layer_2 = relu(np.dot(layer_1,weights_1_2))

    #drop out mask on layer 2
    dropout_mask = np.random.randint(2,size=layer_2.shape)
    layer_2 *= dropout_mask*2

    layer_3 = softmax(np.dot(layer_2,weights_2_3))

    for k in range(batch_size):
      correct_cnt += int(np.argmax(layer_3[k:k+1])== np.argmax(labels[batch_start+k:batch_start+k+1]))
    
    layer_3_delta = (labels[batch_start:batch_end] - layer_3) / (batch_size * layer_3.shape[0])
    layer_2_delta = layer_3_delta.dot(weights_2_3.T) * relu2deriv(layer_2)
    layer_1_delta = layer_2_delta.dot(weights_1_2.T) * tanh2deriv(layer_1)

    # layer_3_delta *= dropout_mask

    weights_2_3 += alpha3 * layer_2.T.dot(layer_3_delta)
    weights_1_2 += alpha2 * layer_1.T.dot(layer_2_delta)
    weights_0_1 += alpha1 * layer_0.T.dot(layer_1_delta)

  test_correct_cnt = 0
  
  for i in range(len(test_images)):
    
    layer_0 = test_images[i:i+1]
    layer_1 = tanh(np.dot(layer_0,weights_0_1))
    layer_2 = relu(np.dot(layer_1,weights_1_2))
    layer_3 = np.dot(layer_2,weights_2_3)

    test_correct_cnt += int(np.argmax(layer_3) == np.argmax(test_labels[i:i+1]))
  
  if(j % 10 == 0):
    print(str(j) + " test_acc" + str(test_correct_cnt/float(len(test_images))) + " Train_acc" + str(correct_cnt/float(len(images))))







0 test_acc0.4683 Train_acc0.16
10 test_acc0.622 Train_acc0.605
20 test_acc0.7388 Train_acc0.712
30 test_acc0.7952 Train_acc0.754
40 test_acc0.8122 Train_acc0.8
50 test_acc0.8296 Train_acc0.817
60 test_acc0.8429 Train_acc0.853
70 test_acc0.85 Train_acc0.866
80 test_acc0.8577 Train_acc0.862
90 test_acc0.8583 Train_acc0.88
100 test_acc0.8625 Train_acc0.901
110 test_acc0.863 Train_acc0.891
120 test_acc0.8667 Train_acc0.893
130 test_acc0.867 Train_acc0.907
140 test_acc0.867 Train_acc0.907
150 test_acc0.8696 Train_acc0.907
160 test_acc0.8712 Train_acc0.911
170 test_acc0.869 Train_acc0.921
180 test_acc0.8695 Train_acc0.927
190 test_acc0.8722 Train_acc0.921
200 test_acc0.8729 Train_acc0.93
210 test_acc0.8714 Train_acc0.939
220 test_acc0.8745 Train_acc0.941
230 test_acc0.8726 Train_acc0.932
240 test_acc0.8745 Train_acc0.939
250 test_acc0.8739 Train_acc0.952
260 test_acc0.8744 Train_acc0.945
270 test_acc0.8756 Train_acc0.954
280 test_acc0.8758 Train_acc0.942
290 test_acc0.8765 Train_acc0.95
